# Green Resilient Supply Chain Network - End-to-End Pipeline

This notebook demonstrates the complete pipeline from data generation to analysis.

## Pipeline Steps:
1. Generate synthetic supply chain data
2. Forecast future demand
3. Build and solve optimization model
4. Run simulation
5. Analyze results and visualize


In [ ]:
import sys
import os

# Add project root to path for imports
# For notebooks, get the current working directory and go up one level if in notebooks/
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'notebooks':
    project_root = os.path.dirname(current_dir)
else:
    project_root = current_dir

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data_gen import generate_supply_chain_data, generate_distance_matrix
from src.forecasting import forecast_demand
from src.pyomo_model import create_supply_chain_model
from src.optimizer import solve_pyomo_model
from src.sim import run_simulation
from src.analysis import analyze_solution, plot_network, plot_demand_forecast, plot_solution_metrics

print("All imports successful!")


## Step 1: Generate Synthetic Supply Chain Data


In [ ]:
# Generate data
data = generate_supply_chain_data(
    n_suppliers=5,
    n_facilities=3,
    n_customers=10,
    n_periods=24,
    seed=42
)

# Generate distance matrices
distances = generate_distance_matrix(
    data['suppliers'],
    data['facilities'],
    data['customers']
)

print(f"Generated data:")
print(f"  - {len(data['suppliers'])} suppliers")
print(f"  - {len(data['facilities'])} facilities")
print(f"  - {len(data['customers'])} customers")
print(f"  - {len(data['demand'])} demand records")

# Display sample data
print("\nSample supplier data:")
print(data['suppliers'].head())
print("\nSample demand data:")
print(data['demand'].head())


## Step 2: Forecast Future Demand


In [ ]:
# Split data into historical and future periods
historical_periods = 18
historical_demand = data['demand'][data['demand']['period'] <= historical_periods].copy()

# Generate forecast
forecast = forecast_demand(
    historical_demand,
    forecast_horizon=6,
    method='moving_average'
)

print(f"Forecasted demand for {len(forecast)} periods")
print("\nSample forecast:")
print(forecast.head())

# Combine historical and forecast for full dataset
full_demand = pd.concat([historical_demand, forecast], ignore_index=True)
print(f"\nTotal demand periods: {len(full_demand)}")


## Step 3: Build and Solve Optimization Model


In [ ]:
# Use only first few periods for faster optimization in demo
demo_demand = full_demand[full_demand['period'] <= 6].copy()

# Create optimization model
model = create_supply_chain_model(
    data['suppliers'],
    data['facilities'],
    data['customers'],
    demo_demand,
    distances
)

print("Model created successfully!")
print(f"  - Suppliers: {len(model.S)}")
print(f"  - Facilities: {len(model.F)}")
print(f"  - Customers: {len(model.C)}")
print(f"  - Periods: {len(model.T)}")


In [ ]:
# Solve the model
print("Solving optimization model...")
result = solve_pyomo_model(model, solver_name='glpk', timeout=30)

print(f"\nSolution status: {result['status']}")
print(f"Termination: {result['termination_condition']}")
print(f"Feasible: {result['feasible']}")

if result['feasible']:
    print(f"Objective value: {result['objective_value']:.2f}")
    if result['solution']:
        print(f"  - {len(result['solution']['flows_sf'])} supplier-facility flows")
        print(f"  - {len(result['solution']['flows_fc'])} facility-customer flows")
        print(f"  - {len(result['solution']['facilities_open'])} facilities opened")
else:
    print("Note: Model may not have solved optimally. Check solver availability.")


## Step 4: Run Simulation


In [ ]:
# Run simulation if we have a solution
if result['feasible'] and result['solution']:
    sim_results = run_simulation(
        data['suppliers'],
        data['facilities'],
        data['customers'],
        demo_demand,
        result['solution'],
        max_time=20.0,
        seed=42
    )
    
    print("Simulation Results:")
    for key, value in sim_results.items():
        print(f"  - {key}: {value}")
else:
    print("Skipping simulation - no feasible solution available")


## Step 5: Analyze Results and Visualize


In [ ]:
# Analyze solution
if result['feasible'] and result['solution']:
    analysis = analyze_solution(
        result['solution'],
        data['suppliers'],
        data['facilities'],
        data['customers'],
        demo_demand
    )
    
    print("Analysis Results:")
    print(f"  - Facilities opened: {analysis.get('n_facilities_opened', 0)}")
    if 'facility_utilization' in analysis:
        print(f"  - Facility utilization: {analysis['facility_utilization']}")


In [ ]:
# Visualize network
if result['feasible'] and result['solution']:
    plot_network(
        data['suppliers'],
        data['facilities'],
        data['customers'],
        solution=result['solution']
    )


In [ ]:
# Plot demand forecast
plot_demand_forecast(historical_demand, forecast)


In [ ]:
# Plot solution metrics
if result['feasible'] and result['solution']:
    plot_solution_metrics(analysis)


## Summary

This notebook demonstrated the complete pipeline:
- ✅ Data generation
- ✅ Demand forecasting  
- ✅ Optimization modeling and solving
- ✅ Simulation
- ✅ Analysis and visualization

All components are working together to create a reproducible supply chain optimization workflow.
